In [1]:
import fileIO
import splice_files
import validate_timestamps
import coordinate_utilities
import time_utilities

In [34]:
# enter name and directory of input file
input_filename1  = 'workingDir/loterie_fixed.gpx'
# input_filename2  = 'workingDir/beep2.gpx'
input_filename2  = 'outputDir/garbagio2.gpx'

output_filename1 = 'outputDir/fixed.gpx'

# select data fields to import
data_flags1 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
data_flags2 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}

In [45]:
# enter name and directory of input file
input_filename1  = 'outputDir/fixed.gpx'
# input_filename2  = 'workingDir/beep2.gpx'
input_filename2  = 'outputDir/garbagio2.gpx'

output_filename1 = 'outputDir/fixed.gpx'

# select data fields to import
data_flags1 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
data_flags2 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}

In [46]:
trkptlist1, meta_data1 = fileIO.import_trackpoints(data_flags1, input_filename1,skip_metadata=True)
trkptlist2, meta_data2 = fileIO.import_trackpoints(data_flags2, input_filename2,skip_metadata=True)

15182 points imported from outputDir/fixed.gpx.
157 points imported from outputDir/garbagio2.gpx.


In [47]:
trkptlist1, meta_data1 = fileIO.import_trackpoints(data_flags1, input_filename1,skip_metadata=True)
# trkptlist2, meta_data2 = fileIO.import_trackpoints(data_flags2, input_filename2,skip_metadata=True)

15182 points imported from outputDir/fixed.gpx.


Generate timestamps for the missing GPX coordinates

In [48]:
gap_times = coordinate_utilities.find_gap(trkptlist=trkptlist1[:13295], mode=1)
print(f"{input_filename1} gap: {gap_times}")

outputDir/fixed.gpx gap: ('20:43:00', '20:51:28')


Check if the timestamps in my gpx log got messed up from the change in timezone:

In [37]:
validate_timestamps.compare_trkpt(trkptlist1,verbose=False)

Fail: 0, Success: 15026


Now we want to identify the gap between Chewbacca rock and the Lottery Farm. This involves finding the index in trkptlist1 where the watch was paused and then unpaused.

In [38]:
gap_times = coordinate_utilities.find_gap(trkptlist=trkptlist1[:13295], mode=1)
print(f"{input_filename1} gap: {gap_times}")
out1 = coordinate_utilities.find_gap(trkptlist=trkptlist1[:13295], mode=2)
print(f"{input_filename1} gap: {out1}")

workingDir/loterie_fixed.gpx gap: ('20:43:00', '20:51:28')
workingDir/loterie_fixed.gpx gap: ((18.07752, -63.070162), (18.080672, -63.071103))


Now find the point in trkptlist2 which is the closest to where the watch was unpaused (i.e. the unpausing timestamp in trkptlist1)

In [39]:
ts = {'year': 2023, 'month': 12, 'day': 21, 'hour': 20, 'minute': 43, 'second': 00}
index = splice_files.get_trackpoint_index_by_time(trkptlist1, ts) # this is the index of the gap of the lottery farm
print("Index of unpausing in trkptlist1: ",index)

Index of unpausing in trkptlist1:  12364


In [40]:
# now find the point in trkptlist2 closest to the lottery farm
out = coordinate_utilities.find_closest(trkptlist1[index],trkptlist2)

trkptlist1_index = index # chebacca rock summit
trkptlist2_index_start = 0 # chebacca rock summit
trkptlist2_index_finish = out-1 #lottery farm

Finding closest point to:  (18.07752,-63.070162)...
Closest point in the list: (18.07728,-63.0698) at index=0


Splicing the files:

In [41]:
trkptlist3 = splice_files.fix_pause_by_index(trkptlist1, trkptlist2, trkptlist1_index, trkptlist2_index_start, trkptlist2_index_finish)

Export the file:

In [42]:
data_flags3 = {
    'gpsflag':  True,
    'timeflag': True,
    'eleflag':  True,
    'hrflag':   True, 
    'cadflag':  False, 
    'pwrflag':  False 
}
fileIO.export_trackpoints(data_flags3, trkptlist3, output_filename1)

Exporting file...
File exported to outputDir/fixed.gpx
